In [ ]:
import numpy as np
import time
import cv2
import os
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from pykalman import KalmanFilter
import pandas as pd

In [ ]:
# display function to show image on Jupyter
def display_img(img,cmap=None):
    fig = plt.figure(figsize = (12,12))
    plt.axis(False)
    ax = fig.add_subplot(111)
    ax.imshow(img,cmap)

In [ ]:
# Load the class labels in which our YOLO model was trained on
labelsPath = os.path.join("class.names")
LABELS = open(labelsPath).read().strip().split("\n")

In [ ]:
# PRint class
LABELS

In [ ]:
# derive the paths to the YOLO weigholov4-custom_finalDS202ts and model configuration
weightsPath = os.path.join("yolov4-custom_last.weights")
configPath = os.path.join("yolov4-custom.cfg")

In [ ]:
# Loading the neural network framework Darknet (YOLO was created based on this framework)
net = cv2.dnn.readNetFromDarknet(configPath,weightsPath)

In [ ]:
# Create the function which predict the frame input
def predict(image):
    
    # initialize a list of colors to represent each possible class label
    np.random.seed(42)
    COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")
    (H, W) = image.shape[:2]
    
    # determine only the "ouput" layers name which we need from YOLO
    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    
    # construct a blob from the input image and then perform a forward pass of the YOLO object detector, 
    # giving us our bounding boxes and associated probabilities
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(ln)
    
    boxes = []
    confidences = []
    classIDs = []
    threshold = 0.2
    
    # loop over each of the layer outputs
    for output in layerOutputs:
        # loop over each of the detections
        for detection in output:
            # extract the class ID and confidence (i.e., probability) of
            # the current object detection
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            # filter out weak predictions by ensuring the detected
            # probability is greater than the minimum probability
            # confidence type=float, default=0.5
            if confidence > threshold:
                # scale the bounding box coordinates back relative to the
                # size of the image, keeping in mind that YOLO actually
                # returns the center (x, y)-coordinates of the bounding
                # box followed by the boxes' width and height
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")

                # use the center (x, y)-coordinates to derive the top and
                # and left corner of the bounding box
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                # update our list of bounding box coordinates, confidences,
                # and class IDs
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)

    # apply non-maxima suppression to suppress weak, overlapping bounding boxes
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, threshold, 0.1)
    #input_h = 1920
    #input_w = 1080
    #laser_line = input_h - 680
    #laser_line_color = (0, 0, 255)
    # ensure at least one detection exists
    if len(idxs) > 0:
        # loop over the indexes we are keeping
        for i in idxs.flatten():
            # extract the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            # draw a bounding box rectangle and label on the image
            color = (255,0,0)
            cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
            text = "{}".format(LABELS[classIDs[i]], confidences[i])
            #cv2.line(img, (0, laser_line), (input_w, laser_line), laser_line_color, 2)
            #cv2.putText(img, "Laser line", (10, laser_line - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, laser_line_color, 2)
            cv2.putText(image, text, (x +15, y - 10), cv2.FONT_HERSHEY_SIMPLEX,
                1, color, 2)
            cv2.imwrite(os.path.join('image' + str(i) + '.jpg'), image)
    return image

In [ ]:
# Execute prediction on a single image
img = cv2.imread('data\image_6563.jpg')
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

In [ ]:
predict(img)

In [ ]:
data = pd.DataFrame(columns = ['X','Y','W','H'])
for i in range(6483,6584):
    img = cv2.imread("data\image_{}.jpg".format(i))
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    a=predict(img)
    df = pd.DataFrame([a], columns = ['X','Y','W','H'])
    data=pd.concat([data,df],ignore_index=True)

In [ ]:
data

In [ ]:
data.to_csv('data.csv')

In [ ]:
def Kalman1D(observations,damping=0.5):
    # To return the smoothed time series data
    observation_covariance = damping
    initial_value_guess = observations[0]
    transition_matrix = 1
    transition_covariance = 0.1
    initial_value_guess
    kf = KalmanFilter(
            initial_state_mean=initial_value_guess,
            initial_state_covariance=observation_covariance,
            observation_covariance=observation_covariance,
            transition_covariance=transition_covariance,
            transition_matrices=transition_matrix
        )
    pred_state, state_cov = kf.smooth(observations)
    return pred_state

In [ ]:
d_smothing = Kalman1D(data['W'].values)
d_smothing

In [ ]:
data["W_smoth"] = d_smothing

In [ ]:
data["%"] = abs((data['W'] - data['W_smoth'])/data['W_smoth'])

In [ ]:
data["W_box"] = data1['W'].values

In [ ]:
data.to_csv("light.csv")

In [ ]:
plt.figure(figsize=(6,4), dpi=120)
plt.xlabel("Frame")
plt.ylabel("Width of bouding box (Pixel)")
plt.plot(d_smothing,'k--',data['W'].values,'k',linewidth=1.5)
plt.savefig("Grap.png")

In [ ]:
Y_center_smothing = Kalman1D(data['Y_center'].values)

In [ ]:
plt.figure(figsize=(6,4), dpi=120)
plt.xlabel("Frame")
plt.ylabel("X_center of bouding box (Pixel)")
plt.plot(data['X_center'].values,'k',linewidth=1.5)
plt.savefig("GrapX_center.png")

In [ ]:
plt.figure(figsize=(6,4), dpi=120)
plt.xlabel("Frame")
plt.ylabel("Y_center of bouding box (Pixel)")
plt.plot(Y_center_smothing,'k--',data['Y_center'].values,'k',linewidth=1.5)
plt.savefig("GrapY_center.png")

In [ ]:
plt.figure(figsize=(6,4), dpi=120)
plt.xlabel("Frame")
plt.ylabel("Width of bouding box predict (Pixel)")
plt.plot(data['W'].values,'k',linewidth=1.5)

In [ ]:
header_list = ['Label','X','Y','W','H']
data1 = pd.DataFrame(columns = ['Label','X','Y','W','H'])
for i in range(6483,6584):
    df1 = pd.read_csv(r'C:\Users\BlueQQ1\Desktop\DenXe\FIleLabel\image_{}.txt'.format(i),sep=' ',header=None,names=header_list)
    data1=pd.concat([data1,df1],ignore_index=True)

In [ ]:
data1['W']=data1['W']*608

In [ ]:
data1['H']=data1['H']*1080

In [ ]:
data1['X']=data1['X']*608-data1['W']/2

In [ ]:
data1['Y']=data1['Y']*1080-data1['H']/2

In [ ]:
data1

In [ ]:
data1= data1[data1.X>117]

In [ ]:
data1 = data1[data1.X<260]

In [ ]:
data1['X']

In [ ]:
plt.figure(figsize=(6,4), dpi=120)
plt.xlabel("Frame")
plt.ylabel("Width of bouding box (Pixel)")
plt.plot(data['W'].values,'k',linewidth=1)
plt.plot(d_smothing,'k',linestyle = 'dotted',linewidth=1.5)
plt.plot(box.values,'k--',linewidth=1)
plt.show()

In [ ]:
data1["W"].values

In [ ]:
data